In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pandas_profiling import ProfileReport
import pickle
import os
import datetime
import calendar
from scipy.stats import zscore

In [2]:
html_folder = 'C:\\Users\\Brayden\\Desktop\\Personal Website\\Brayden-L.github.io\\_includes\\creek_raptors\\'

In [3]:
# Upload and clean
df, _, _ = pickle.load(open('C:/Users/Brayden/Desktop/LocScrapes/Indian Creek (Full).pkl', 'rb'))
df.drop_duplicates(subset='URL', inplace=True)
df['Base Location'] = df['Base Location'].apply(lambda x: x.strip())

In [4]:
r_walls_1 = ['Suburbia', 'Echoes Wall', 'The Wall', 'Far Side', 'Cliffs of Insanity', 'Original Meat Wall', 'Tenderloins', '2nd Meat Wall', '1st Meat Wall', 'Disappointment Cliffs', 'Selfish Wall', 'Shock and Awe', 'Public Service Wall', 'Fin Wall', 'Broken Tooth', 'Cat Wall', 'Slug Wall', 'Reservoir Wall']
r_walls_2 = ['The Wall', 'Far Side', 'Original Meat Wall', 'Tenderloins', '2nd Meat Wall', '1st Meat Wall', 'Disappointment Cliffs', 'Fin Wall', 'Broken Tooth', 'Cat Wall', 'Slug Wall', 'Reservoir Wall']
r_walls_3 = ['The Wall', 'Far Side','2nd Meat Wall', 'Disappointment Cliffs', 'Fin Wall', 'Broken Tooth', 'Cat Wall', 'Slug Wall', 'Reservoir Wall']
r_walls_4 = ['The Wall', 'Cat Wall', 'Reservoir Wall']

df_rapt_restr = pd.DataFrame({'Year':['2023', '2022', '2021', '2020', '2019', '2018'], 
                              'Start Date': ['2023-2-27', '2022-3-1', '2021-2-11', '2020-2-28', '2019-2-14', '2018-3-1'], 
                              'End Date': [None, '2022-8-19', '2021-9-2', '2020-9-1', '2019-9-3', '2018-8-31'], 
                              'Restricted Walls': [r_walls_1, r_walls_1, r_walls_1, r_walls_2, r_walls_3, r_walls_4]
                              })
df_rapt_restr['Start Date'] = pd.to_datetime(df_rapt_restr['Start Date'])
df_rapt_restr['End Date'] = pd.to_datetime(df_rapt_restr['End Date'])

The basic plan is to:
1. Create new columns for route name and route base location in each routes tick dataframe.
2. Iterate over each year for raptor restrictions.
3. Filter for only routes that are in raptor restriction zones.
4. Concatenate all tick data that exists in the restriction zones.
5. Filter by date to see if tick occurred during restriction.

In [5]:
for index, row in df.iterrows():
    row['Route Ticks']['Route'] = row['Route']
    row['Route Ticks']['Base Location'] = row['Base Location']
    row['Route Ticks']['Rating'] = row['Rating']

In [6]:
df_super_tick_list = pd.concat(df['Route Ticks'].to_list())
df_super_tick_list.loc[df_super_tick_list['Username']=='', 'Username'] = 'Private'

In [7]:
# df_super_tick_list = df_super_tick_list[df_super_tick_list['Date'].dt.month == 8]

In [8]:
df_rapt_bb = pd.DataFrame()
df_rapt_bb_alldates_restrwalls = pd.DataFrame()
df_rapt_bb_allwalls_restrdates = pd.DataFrame()
for index, row in df_rapt_restr.iterrows():
    restr_walls = row['Restricted Walls']
    start_date = row['Start Date']
    end_date = row['End Date']

    df_tick_bb_sub1 = df_super_tick_list[(df_super_tick_list['Base Location'].isin(restr_walls)) & (start_date <= df_super_tick_list['Date']) & (df_super_tick_list['Date'] <= end_date)]
    df_rapt_bb = pd.concat([df_rapt_bb, df_tick_bb_sub1])
    
    df_tick_bb_sub2 = df_super_tick_list[(df_super_tick_list['Base Location'].isin(restr_walls))]
    df_rapt_bb_alldates_restrwalls = pd.concat([df_rapt_bb_alldates_restrwalls, df_tick_bb_sub2])
    
    df_tick_bb_sub3 = df_super_tick_list[(start_date <= df_super_tick_list['Date']) & (df_super_tick_list['Date'] <= end_date)]
    df_rapt_bb_allwalls_restrdates = pd.concat([df_rapt_bb_allwalls_restrdates, df_tick_bb_sub3])
df_rapt_bb

,Username,User Link,Date,Pitches Ticked,Style,Lead Style,Comment,Route,Base Location,Rating
61,Amanda Gibson,/user/7077573,2022-06-16,1.0,Lead,Onsight,Super fun warm up.,Dr. Carl,Reservoir Wall,5.10-
62,Allison Saltz,/user/200407991,2022-04-15,1.0,,,,Dr. Carl,Reservoir Wall,5.10-
63,Kalin Roethle,/user/201338041,2022-04-04,1.0,Follow,,,Dr. Carl,Reservoir Wall,5.10-
64,Private,,2022-03-24,1.0,,,,Dr. Carl,Reservoir Wall,5.10-
40,Private,,2022-03-24,1.0,,,,Kelley Route 29 aka Three Fools,Reservoir Wall,5.10
...,...,...,...,...,...,...,...,...,...,...
34,L S,/user/201477006,2018-03-18,1.0,,,,Burl Dog,Cat Wall,5.12+
134,Adam Riser,/user/200294925,2018-08-15,1.0,,,,Sorrow,The Wall,5.11
135,Brian Williams,/user/200242299,2018-04-01,1.0,TR,,"one of my favorites from the Oct BKBX trip, na...",Sorrow,The Wall,5.11
213,Brian Williams,/user/200242299,2018-04-01,1.0,TR,,,Pigs On The Wing,The Wall,5.11


## By Year

In [9]:
rapt_year_bb = df_rapt_bb['Date'].groupby(df_rapt_bb['Date'].dt.year).count()

fig = px.bar(rapt_year_bb, text_auto=True, height=300)
fig.update_xaxes(title='Year')
fig.update_yaxes(title='Num Ticks')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(title='Sum Raptor Violations by Year | 2018-2022', title_x=0.5, showlegend=False, plot_bgcolor='#EEE3DD')
fig.write_html(html_folder + 'year_viol.html')
fig

In [10]:
rapt_year_all = df_rapt_bb_alldates_restrwalls['Date'].groupby(df_rapt_bb_alldates_restrwalls['Date'].dt.year).count()
rapt_year_all = rapt_year_all.loc[rapt_year_bb.index]

fig = px.bar(rapt_year_all, text_auto='0', height=300)
fig.update_xaxes(title='Year')
fig.update_yaxes(title='Num Ticks')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(title='Sum Ticks at Walls of Interest by Year | 2018-2022', title_x=0.5, showlegend=False, plot_bgcolor='#EEE3DD')
fig.write_html(html_folder + 'year_tot.html')
fig

In [11]:
# This is the percent of ticks at restricted walls that occurred during raptor restrictions. This normalizes by overall year activity at these walls which decouples year over year variation for these walls.
rapt_year_norm = (rapt_year_bb/rapt_year_all)*100

fig = px.bar(rapt_year_norm, text_auto='.2', height=300)
fig.update_xaxes(title='Year')
fig.update_yaxes(title='Percent (%)')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(title='Normalized Raptor Violations by Year | Ratio of Ticks of Raptor Violations Over All Ticks at Walls of Interest | 2018-2022', title_x=0.5, showlegend=False, plot_bgcolor='#EEE3DD', title_font_size=12)
fig.write_html(html_folder + 'year_norm.html')
fig
# This suggests that 2018, 2020, 2022 were good years, and 2019 and 2021 were bad years.

## By Month

In [12]:
rapt_month_bb = df_rapt_bb['Date'].groupby(df_rapt_bb['Date'].dt.month).count()
rapt_month_bb.index = rapt_month_bb.index.map({1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June', 7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'})

fig = px.bar(rapt_month_bb, text_auto='0', height=300)
fig.update_xaxes(title='Month')
fig.update_yaxes(title='Num Ticks')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(title='Sum Raptor Violations by Month | 2018-2022', title_x=0.5, showlegend=False, plot_bgcolor='#EEE3DD')
fig.write_html(html_folder + 'month_viol.html')
fig

In [13]:
# This is total ticks at all walls during that restricted timeframe
rapt_month_all = df_rapt_bb_allwalls_restrdates['Date'].groupby(df_rapt_bb_allwalls_restrdates['Date'].dt.month).count()
rapt_month_all
rapt_month_all.index = rapt_month_all.index.map({1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June', 7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'})
rapt_month_all = rapt_month_all.loc[rapt_month_bb.index]

fig = px.bar(rapt_month_all, text_auto='0', height=300)
fig.update_xaxes(title='Month')
fig.update_yaxes(title='Num Ticks')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(title='Sum Ticks at All Walls by Month | 2018-2022', title_x=0.5, showlegend=False, plot_bgcolor='#EEE3DD')
fig.write_html(html_folder + 'month_tot.html')
fig

In [14]:
# This is the ratio of violations at restricted walls to ticks at all walls for each month over the year range 2018-2022. This normalizes by overall month activity at ALL walls which attempts to decouple month to month variation.
rapt_month_norm = (rapt_month_bb/rapt_month_all)*100

fig = px.bar(rapt_month_norm, text_auto='.2', height=300)
fig.update_xaxes(title='Month')
fig.update_yaxes(title='Percent (%)')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(showlegend=False, plot_bgcolor='#EEE3DD', title='Normalized Raptor Violations by Month | Ratio of Ticks That Were Violations Over Ticks at All Walls During Restricted Timeframe | 2018-2022', title_x=0.5, title_font_size=12)
fig.write_html(html_folder + 'month_norm.html')
fig
# This suggests compliance during off-season is worse than high-season. Perhaps due to lower steward presence? February is a particular outlier. Is the word not getting out fast enough?

## By Crag

In [15]:
rapt_wall_bb = df_rapt_bb.groupby('Base Location').count()['Route'].sort_values(ascending=False)

fig = px.bar(rapt_wall_bb, text_auto='0', height=300)
fig.update_xaxes(title='Crag')
fig.update_yaxes(title='Num Ticks')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(title='Sum Raptor Violations by Crag | 2018-2022', title_x=0.5, showlegend=False, plot_bgcolor='#EEE3DD')
fig.write_html(html_folder + 'crag_viol.html')
fig

In [16]:
rapt_wall_all = df_rapt_bb_alldates_restrwalls.groupby('Base Location').count()['Route'].reindex(rapt_wall_bb.index)

fig = px.bar(rapt_wall_all, text_auto='0', height=300)
fig.update_xaxes(title='Crag')
fig.update_yaxes(title='Num Ticks')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(title='Sum Ticks by Crag | 2018-2022', title_x=0.5, title_font_size=12, showlegend=False, plot_bgcolor='#EEE3DD')
fig.write_html(html_folder + 'crag_tot.html')
fig

In [17]:
# Normalize by all ticks year around at wall of interest to normalize by crag popularity
rapt_wall_norm = (rapt_wall_bb/rapt_wall_all)*100

fig = px.bar(rapt_wall_norm, text_auto='.2', height=300)
fig.update_xaxes(title='Crag')
fig.update_yaxes(title='Percent (%)')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(showlegend=False, plot_bgcolor='#EEE3DD', title='Normalized Raptor Violations by Crag | Ratio of Raptor Violations Over All Ticks at Walls of Interest | 2018-2022', title_x=0.5, title_font_size=12)
fig.write_html(html_folder + 'crag_norm.html')
fig
# This suggests that Selfish Wall and 2nd Meat Wall are more violated given their popularity. All of the lesser traveled are particularly violated, with Suburbia being a concerning one due to its higher volume.

## By Route

In [18]:
rapt_route_bb = df_rapt_bb.groupby('Route').count()['Username'].sort_values(ascending=False)

fig = px.bar(rapt_route_bb[0:30], height=300)
fig.update_xaxes(title='Route')
fig.update_yaxes(title='Num Ticks')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(showlegend=False, plot_bgcolor='#EEE3DD', title='Raptor Violation Count by Route Top 30 | 2018-2022', title_x=0.5, font_size=10)
fig.write_html(html_folder + 'route_viol.html')
fig

In [19]:
rapt_route_all = df_rapt_bb_alldates_restrwalls[df_rapt_bb_alldates_restrwalls['Route'].isin(rapt_route_bb.index.to_list())] # First we want to filter for only routes that have a violation.
rapt_route_all = rapt_route_all[rapt_route_all['Date'].dt.year.isin([2018, 2019, 2020, 2021, 2022])] # Then we filter down to ticks that occur in our years of interest.
rapt_route_all = rapt_route_all.groupby('Route').count()['Username'].reindex(rapt_route_bb.index.to_list()).sort_values(ascending=False)

fig = px.bar(rapt_route_all[0:30], height=300)
fig.update_xaxes(title='Route')
fig.update_yaxes(title='Num Ticks')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(showlegend=False, plot_bgcolor='#EEE3DD', title='Sum Ticks By Route Top 30 | 2018-2022', title_x=0.5, font_size=10)
fig.write_html(html_folder + 'route_tot.html')
fig

In [20]:
rapt_route_all_filt = rapt_route_all[rapt_route_all>30]
rapt_route_norm = rapt_route_bb*(rapt_route_bb/rapt_route_all_filt)
rapt_route_norm.sort_values(ascending=False, inplace=True)

fig = px.bar(rapt_route_norm[0:30], height=300)
fig.update_xaxes(title='Route')
fig.update_yaxes(title='Percent (%)')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(showlegend=False, plot_bgcolor='#EEE3DD', title='Normalized Raptor Violations by Route Top 30 | Ratio of Raptor Violations Over All Ticks on Route of Interest | Ignored <30 Ticks | 2018-2022', title_x=0.5, title_font_size=10, font_size=10)
fig.write_html(html_folder + 'route_norm.html')
fig
# Normalization here has a squared term to prioritize higher ticks but still consider normalization. This is required otherwise routes with few ticks would overwhelm.

# Rating

In [21]:
rapt_rating_bb = df_rapt_bb.groupby('Rating').count()['Username'].sort_values(ascending=False)

rapt_rating_beginner = sum(rapt_rating_bb[['5.8', '5.8+', '5.9-', '5.9', '5.9+']])
rapt_rating_intermediate = sum(rapt_rating_bb[['5.10-', '5.10', '5.10+', '5.10a', '5.10b', '5.10c', '5.10d']])
rapt_rating_advanced = sum(rapt_rating_bb[['5.11-', '5.11', '5.11+', '5.11a', '5.11b', '5.11c', '5.11d', '5.11b/c', '5.11a/b']])
rapt_rating_expert = sum(rapt_rating_bb[['5.12-', '5.12', '5.12+', '5.13-']])

rapt_rating_bb_summ = pd.Series(data=[rapt_rating_beginner, rapt_rating_intermediate, rapt_rating_advanced, rapt_rating_expert], index=['<=5.9', '5.10', '5.11', '>=5.12'])

fig = px.bar(rapt_rating_bb_summ, text_auto='0', height=300)
fig.update_xaxes(title='Grade')
fig.update_yaxes(title='Num Ticks')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(showlegend=False, plot_bgcolor='#EEE3DD', title='Raptor Violation Count by Grade | 2018-2022', title_x=0.5)
fig.write_html(html_folder + 'grade_viol.html')
fig

In [22]:
df_super_tick_list_limited_years = df_super_tick_list[df_super_tick_list['Date'].dt.year.isin([2018, 2019, 2020, 2021, 2022])]
rapt_rating_all = df_super_tick_list_limited_years.groupby('Rating').count()['Username']

rapt_rating_beginner = sum(rapt_rating_all[['5.8', '5.8+', '5.9-', '5.9', '5.9+']])
rapt_rating_intermediate = sum(rapt_rating_all[['5.10-', '5.10', '5.10+', '5.10a', '5.10b', '5.10c', '5.10d']])
rapt_rating_advanced = sum(rapt_rating_all[['5.11-', '5.11', '5.11+', '5.11a', '5.11b', '5.11c', '5.11d', '5.11b/c', '5.11a/b']])
rapt_rating_expert = sum(rapt_rating_all[['5.12-', '5.12', '5.12+', '5.13-']])

rapt_rating_all_summ = pd.Series(data=[rapt_rating_beginner, rapt_rating_intermediate, rapt_rating_advanced, rapt_rating_expert], index=['<=5.9', '5.10', '5.11', '>=5.12'])

fig = px.bar(rapt_rating_all_summ, text_auto='0', height=300)
fig.update_xaxes(title='Grade')
fig.update_yaxes(title='Num Ticks')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(showlegend=False, plot_bgcolor='#EEE3DD', title='Sum Ticks by Grade | 2018-2022', title_x=0.5)
fig.write_html(html_folder + 'grade_tot.html')
fig

In [23]:
rapt_rating_norm = ((rapt_rating_bb_summ)/rapt_rating_all_summ)*100

fig = px.bar(rapt_rating_norm, text_auto='.2', height=300)
fig.update_xaxes(title='Grade')
fig.update_yaxes(title='Percent (%)')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(showlegend=False, plot_bgcolor='#EEE3DD', title='Normalized Raptor Violations by Grade | Ratio of Raptor Violations Over All Ticks for All Routes | 2018-2022', title_x=0.5, title_font_size=12)
fig.write_html(html_folder + 'grade_norm.html')
fig

## Violator Analysis

In [24]:
print(f'Number of Violations: {df_rapt_bb.shape[0]}')
print(f'Number of Violators: {df_rapt_bb["Username"].unique().shape[0]}')

Number of Violations: 1070
Number of Violators: 325


In [25]:
rapt_violator_vc = df_rapt_bb['Username'].value_counts().value_counts()

fig = px.bar(rapt_violator_vc, height=300)
fig.update_xaxes(title='Number of Violations')
fig.update_yaxes(title='Count')
fig.update_traces(marker_color='#F4A460')
fig.update_layout(showlegend=False, plot_bgcolor='#EEE3DD', title='Raptor Violations by User Value Count', title_x=0.5)
fig.write_html(html_folder + 'violator_value_count.html')
fig

In [26]:
rapt_violator = df_rapt_bb['Username'].value_counts()

fig = px.bar(rapt_violator[0:30], title='Raptor Violations by User Top 50')
fig
# A cursory glance at higher offenders show that they are typically either strong or high volume climbers, AKA they should know better. How to quantify this?

In [52]:
df_repeat_off_day = df_rapt_bb.groupby(['Username'])['Date'].count().value_counts().sort_index()
df_repeat_off_day.drop(df_repeat_off_day.index[-1], inplace=True)
px.bar(df_repeat_off_day, title='Count of Users Who Violated N Days')

In [44]:
df_repeat_off_day_bad = df_repeat_off_day[df_repeat_off_day>=5]
rapt_violator[rapt_violator.index.isin(df_repeat_off_day_bad.index.to_list())]

Series([], Name: Username, dtype: int64)

In [27]:
rapt_repeat_off_year = df_rapt_bb.groupby(['Username', df_rapt_bb['Date'].dt.year]).count()['Date'].groupby('Username').unique().apply(lambda x: len(x)).sort_values(ascending=False)
rapt_repeat_off_year.value_counts()
# Looks like most people do not violate across more than one year

1    310
2     14
4      1
Name: Date, dtype: int64

In [28]:
rapt_repeat_off_2 = rapt_repeat_off_year.loc[rapt_repeat_off_year==2]
rapt_repeat_off_2

Username
Britt Z              2
Colten L             2
BronsonH             2
Markus Reitenbach    2
Holly H              2
Gregarious Poodle    2
Brett B              2
Maggie Keating       2
Gaby Colletta        2
Fearless 1           2
Chris Lehan          2
A Johnston           2
Raudel Hernandez     2
tshapiro1182         2
Name: Date, dtype: int64

In [29]:
rapt_violator[rapt_violator.index.isin(rapt_repeat_off_2.index.to_list())]
# Violators tend to be experienced climbers, not beginner climbers. Caveat that this is biased to people who tick on MP, who are more likely to be experienced climbers.

Fearless 1           18
Gaby Colletta        13
A Johnston           11
tshapiro1182         11
Maggie Keating        9
Brett B               9
Raudel Hernandez      9
Gregarious Poodle     7
Britt Z               5
Holly H               4
BronsonH              4
Colten L              4
Chris Lehan           4
Markus Reitenbach     3
Name: Username, dtype: int64